# How do I _copy_ a file from another on of _my projects_?
### Overview
Files can be copied from one of two sources:
1. other projects you are a member of
2. public reference.  

Here we focus on copying a file from _your own project_. We will show two options (A & B) of achieving the same goal. 

### Prerequisites
 1. You need to be a member (or owner) of _at least **two**_ projects.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="files_listAll.ipynb" target="_blank">list</a> files within one of your projects (we will just use that call directly here).
 
### Imports and Definitions
A single call is sufficient to get a file list. We will show two different options, both of which are defined in the apimethods.py file. Here there is **no advantage** in using the object to POST the call.
 
### Warning
You are only permitted to copy a TCGA-protected file to a _target project_ if that project also has TCGA-protected permissions.

In [ ]:
from defs.apimethods import *

## (Option A) Use API() _object_
We will first list all our projects, then list the files within the first project, and copy a file between the first and second project.

The critical information for this POST is the **file_id**. Note, you are allow to copy the same file as many times as you like. However, duplicates will be automatically have a prefix attached of (\_1\_, \_2\_, etc) depending on how many times you copy the file.

To make these results very obvious, use an empty project as your TARGET\_PROJECT (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome.copy'

In [ ]:
# [USER INPUT] Set project name; project (p_) and file (f_) indices here:
f_index = 0                               # file to copy
p_index_source = 1                        # source project
p_index_target = 0                        # target project

# LIST all projects
existing_projects = API('projects')         

# LIST all files in the source and target project
my_files_source = API(path='files', \
                      query={'project':existing_projects.id[p_index_source], \
                            'limit':100})
my_files_target = API(path='files', \
                      query={'project': existing_projects.id[p_index_target]})

# Check if first file already exists in the target project
if my_files_source.name[f_index] in my_files_target.name:
    print('file already exists in second project, please try another file')
else:
    print('File (%s) does not exist in Project (%s); copying now' % \
          (my_files_source.name[f_index], existing_projects.id[p_index_target]))
    
    # COPY the selected file from source to target project
    API(path=('files/' + my_files_source.id[f_index] + '/actions/copy'), \
        method='POST', \
        data={'project': existing_projects.id[p_index_target],\
              'name': my_files_source.name[f_index]}) 

    # re-list files in target project to verify the copy worked
    my_files_target = API(path='files', \
                          query={'project': existing_projects.id[p_index_target]})
    
    if my_files_source.name[f_index] in my_files_target.name:
        print('Sucessfully copied one file!')
    else:
        print('Something went wrong...')

## (Option B) Use api_call() *function*
Using the api_call() to do the same thing as above, except that here we do not check for the file already existing in the second project.

**Note** by using api_call() without assigning it to a variable, we print the response dictionary from the call. 

In [ ]:
# [USER INPUT] Set project name; project (p_) and file (f_) indices here:
f_index = 0                               # file to copy
p_index_source = 1                        # source project
p_index_target = 0                        # target project

# Short (n=25) list of projects I am a member of
existing_projects = api_call(path='projects', method='GET', \
                            query={'limit':25, 'offset':0}, data=None)      

# LIST (up to) 100 files in source and target project
my_files_source = api_call(path='files', \
                           query={'limit':100, \
                                  'project': existing_projects['items'][p_index_source]['id']})
my_files_target = api_call(path='files', \
                           query={'limit':100, \
                                  'project': existing_projects['items'][p_index_target]['id']})

# COPY the selected file from source to target project
api_call(path=('files/' + my_files_source['items'][f_index]['id'] + '/actions/copy'), \
    method='POST', \
    data={'project': existing_projects['items'][p_index_target]['id'],\
          'name': 'Dept of Awesome.copy'}) 

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/copy-a-file)